In [1]:
cd D:\Algorithmic Marketing\Assignment 3\recommenders

D:\Algorithmic Marketing\Assignment 3\recommenders


In [5]:
import sys
sys.path.append("../../")
import os
import surprise
import papermill as pm
import pandas as pd

from reco_utils.common.timer import Timer
from reco_utils.dataset import movielens
from reco_utils.dataset.python_splitters import python_random_split
from reco_utils.evaluation.python_evaluation import (rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, precision_at_k, 
                                                     recall_at_k, get_top_k_items)
from reco_utils.recommender.surprise.surprise_utils import predict, compute_ranking_predictions

print("System version: {}".format(sys.version))
print("Surprise version: {}".format(surprise.__version__))

System version: 3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]
Surprise version: 1.1.1


In [37]:
data = pd.read_csv("D:/Algorithmic Marketing/Final Project/Recommendation_data.csv")

In [38]:
data.head(10)

,userID,itemID,rating
0,17850,85123A,8
1,17850,71053,7
2,17850,84406B,8
3,17850,84029G,5
4,17850,84029E,9
5,17850,22752,7
6,17850,21730,9
7,17850,22633,6
8,17850,22632,5
9,17850,22632,7


In [39]:
train, test = python_random_split(data, 0.75)

In [40]:
train_set = surprise.Dataset.load_from_df(train, reader=surprise.Reader('ml-100k')).build_full_trainset()
train_set

In [41]:
svd = surprise.SVD(random_state=0, n_factors=200, n_epochs=30, verbose=True)

with Timer() as train_time:
    svd.fit(train_set)

print("Took {} seconds for training.".format(train_time.interval))

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29
Took 98.4776812 seconds for training.


In [42]:
predictions = predict(svd, test, usercol='userID', itemcol='itemID')
predictions.head()

,userID,itemID,prediction
0,15529,22326,5.0
1,15156,22045,5.0
2,15660,22423,5.0
3,15189,22131,5.0
4,15640,22741,5.0


In [43]:
#Removing products the customer has already bought
with Timer() as test_time:
    all_predictions = compute_ranking_predictions(svd, train, usercol='userID', itemcol='itemID', remove_seen=True)
    
print("Took {} seconds for prediction.".format(test_time.interval))

Took 361.99908449999975 seconds for prediction.


In [44]:
all_predictions.head()

,userID,itemID,prediction
297216,14665,20727,5.0
297217,14665,21931,5.0
297218,14665,22080,5.0
297219,14665,21340,5.0
297220,14665,48187,5.0


In [45]:
eval_rmse = rmse(test, predictions)
eval_mae = mae(test, predictions)
eval_rsquared = rsquared(test, predictions)
eval_exp_var = exp_var(test, predictions)

k = 10
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=k)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=k)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=k)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=k)


print("RMSE:\t\t%f" % eval_rmse,
      "MAE:\t\t%f" % eval_mae,
      "rsquared:\t%f" % eval_rsquared,
      "exp var:\t%f" % eval_exp_var, sep='\n')

print('----')

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

RMSE:		2.451093
MAE:		2.000027
rsquared:	-1.992368
exp var:	-0.000013
----
MAP:	0.002902
NDCG:	0.016522
Precision@K:	0.013356
Recall@K:	0.008077
